# Piper TTS Model Export

This notebook exports your trained Piper TTS model to ONNX format for inference.

## Prerequisites
- Completed at least 1 epoch of training
- Checkpoint file exists in `training_dir/lightning_logs/`

### 1. Setup Environment

In [23]:
import os
import sys
from pathlib import Path
import glob

# Set paths
piper_src_path = os.path.abspath("../piper_repo/src/python")
training_dir_ruru = os.path.abspath("training_dir_reva")
export_dir = os.path.abspath("exported_models")

# Create export directory
os.makedirs(export_dir, exist_ok=True)

print(f"Piper Source: {piper_src_path}")
print(f"Training Dir: {training_dir_ruru}")
print(f"Export Dir: {export_dir}")

Piper Source: /Users/rutwik/piper-model-training/piper_repo/src/python
Training Dir: /Users/rutwik/piper-model-training/scripts/training_dir_reva
Export Dir: /Users/rutwik/piper-model-training/scripts/exported_models


### 2. Find Latest Checkpoint

In [24]:
# Find all checkpoint files
checkpoint_pattern = os.path.join(training_dir_ruru, "lightning_logs/*/checkpoints/*.ckpt")
checkpoints = glob.glob(checkpoint_pattern)

if not checkpoints:
    print("❌ No checkpoints found!")
    print(f"Searched in: {checkpoint_pattern}")
    print("\nMake sure you've completed at least 1 epoch of training.")
else:
    # Sort by modification time, get latest
    latest_checkpoint = max(checkpoints, key=os.path.getmtime)
    checkpoint_name = Path(latest_checkpoint).stem
    
    print(f"✅ Found {len(checkpoints)} checkpoint(s)")
    print(f"\nLatest checkpoint: {checkpoint_name}")
    print(f"Full path: {latest_checkpoint}")

✅ Found 1 checkpoint(s)

Latest checkpoint: epoch=1015-step=96
Full path: /Users/rutwik/piper-model-training/scripts/training_dir_reva/lightning_logs/version_0/checkpoints/epoch=1015-step=96.ckpt


### 3. Export to ONNX Format

This exports the model using the `piper_train.export_onnx` module.

In [25]:

if not checkpoints:
    print("⚠️ Skipping export - no checkpoint available")
else:
    # Define output paths
    # output_model = f'checkpoints/epoch=2519-step=9120.ckpt.onnx'
    # output_model = os.path.join(export_dir, f"{checkpoint_name}.onnx")
    output_model = os.path.join(export_dir, f"reva.onnx")

    latest_checkpoint = f'training_dir_reva/lightning_logs/version_0/checkpoints/epoch=1015-step=96.ckpt'
    output_config = f"{output_model}.json"
    
    print(f"Exporting model to: {output_model}")
    print("This may take a few minutes...\n")
    
    # Run export command
    !PYTHONPATH="{piper_src_path}" "{sys.executable}" -m piper_train.export_onnx \
        "{latest_checkpoint}" \
        "{output_model}"
    
    # Copy config file
    config_source = os.path.join(training_dir_ruru, "config.json")
    if os.path.exists(config_source):
        import shutil
        shutil.copy(config_source, output_config)
        print(f"\n✅ Export complete!")
        print(f"\nModel files:")
        print(f"  - {output_model}")
        print(f"  - {output_config}")
    else:
        print(f"\n⚠️ Warning: Could not find config.json at {config_source}")

Exporting model to: /Users/rutwik/piper-model-training/scripts/exported_models/reva.onnx
This may take a few minutes...

/Users/rutwik/miniconda3/envs/voice_training/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Removing weight norm...
/Users/rutwik/piper-model-training/piper_repo/src/python/piper_train/vits/attentions.py:235: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  t_s == t_t
/Users/rutwik/piper-model-training/piper_repo/src/python/piper_train/vits/attentions.py:295: TracerWarning: Converting a tensor to a Python boolean 

In [27]:
# Test if piper is installed
import subprocess
import os
import site

# 1. Add user bin to PATH (just in case)
try:
    user_bin = os.path.join(site.getuserbase(), 'bin')
    if user_bin not in os.environ['PATH']:
        os.environ['PATH'] += os.pathsep + user_bin
except:
    pass

# 2. Check for Piper CLI (Updated to accept exit code 2)
try:
    # check=False allows non-zero exit codes (like missing arguments) without crashing
    result = subprocess.run(["piper", "--help"], check=False, capture_output=True, text=True)
    
    # If we got ANY output or a specific exit code, it's installed
    if result.returncode in [0, 2] or "usage: piper" in result.stderr:
        piper_available = True
        print("✅ Piper CLI is available")
    else:
        piper_available = False
        print("ℹ️ Piper CLI not found")
        print(f"Debug: Return code {result.returncode}")
        print(f"Debug Output: {result.stderr}")

except FileNotFoundError:
    piper_available = False
    print("ℹ️ Piper CLI not found (FileNotFoundError)")
    print("Install with: pip install piper-tts")

✅ Piper CLI is available


### 4. Test Exported Model (Optional)

If you have Piper CLI installed, you can test the exported model.

In [1]:
import os

# Ensure the config file exists (I've already created it for you)
# /Users/rutwik/piper-model-training/scripts/exported_models/epoch=1075-step=456.onnx
#   - /Users/rutwik/piper-model-training/scripts/exported_models/epoch=1075-step=456.onnx.json
model_path = './exported_models/reva.onnx'
config_path = './exported_models/reva.onnx.json'

if not os.path.exists(config_path):
    print(f"Warning: {config_path} not found. Piper requires a .json config file with the same name as the model.")

test_text = """ Prachi how is your World"""
test_output = "./exported_models/test_01.wav"

print(f"Testing with text: '{test_text}'")
print(f"Model path: {model_path}")
print(f"config path: {config_path}")
!echo "{test_text}" | piper -m "{model_path}" --output_file "{test_output}"

if os.path.exists(test_output):
    print(f"\n✅ Test audio generated: {test_output}")
    try:
        from IPython.display import Audio, display
        display(Audio(test_output))
    except ImportError:
        pass
else:
    print("\n❌ Test failed - no audio file generated")

Testing with text: ' Prachi how is your World'
Model path: ./exported_models/reva.onnx
config path: ./exported_models/reva.onnx.json

✅ Test audio generated: ./exported_models/test_01.wav


### 5. Model Information

In [8]:
if checkpoints:
    import json
    
    # Display model info
    print("="*60)
    print("EXPORTED MODEL INFORMATION")
    print("="*60)
    print(f"\nCheckpoint: {checkpoint_name}")
    print(f"Model file: {output_model}")
    print(f"Config file: {output_config}")
    
    # Get file sizes
    if os.path.exists(output_model):
        model_size_mb = os.path.getsize(output_model) / (1024 * 1024)
        print(f"\nModel size: {model_size_mb:.2f} MB")
    
    # Show config
    if os.path.exists(output_config):
        with open(output_config, 'r') as f:
            config = json.load(f)
        print(f"\nConfig details:")
        print(f"  Sample rate: {config.get('audio', {}).get('sample_rate', 'N/A')} Hz")
        print(f"  Num speakers: {config.get('num_speakers', 'N/A')}")
        print(f"  Num symbols: {config.get('num_symbols', 'N/A')}")
    
    print("\n" + "="*60)
    print("\n✅ Export complete! You can now use this model with Piper.")
    print(f"\nUsage example:")
    print(f'  echo "Your text here" | piper -m {output_model} --output_file output.wav')

EXPORTED MODEL INFORMATION

Checkpoint: epoch=1500-step=3006
Model file: /Users/rutwik/piper-model-training/scripts/exported_models/rutwik.onnx
Config file: /Users/rutwik/piper-model-training/scripts/exported_models/rutwik.onnx.json

Model size: 60.59 MB

Config details:
  Sample rate: 22050 Hz
  Num speakers: 1
  Num symbols: 256


✅ Export complete! You can now use this model with Piper.

Usage example:
  echo "Your text here" | piper -m /Users/rutwik/piper-model-training/scripts/exported_models/rutwik.onnx --output_file output.wav
